In [1]:
import numpy as np
from tqdm import tqdm_notebook

from tools.data_loading import load_images, load_labels, dummy_code, load_images_resized
from tools.feature_learning import pins_generation
from tools.hog import hog
from tools.kernels import kernel_matrix
from tools.optimization import find_f
from tools.prediction import pred
from tools.process_images import process_images
from tools.quantization import kmeans
from tools.submission import labels_to_csv
from tools.visualization import imshow, dump_as_png, reshape_as_images

# Data loading

In [2]:
X_train = load_images_resized(type="train")
n_train = X_train.shape[0]
Y_labels_train = load_labels()
Y_train = dummy_code(Y_labels_train)
n_classes = Y_train.shape[1]
X_train = X_train - X_train.min(axis=0)
X_train = X_train / X_train.max(axis=0)

In [3]:
X_train2 = np.zeros((5000,63,63,3))
for i in range(3):
    X_train2[:,:,:,i] = X_train
image_list = X_train2

# Visual features

In [4]:
filter_sigma=0.1
filter_shape=5
hog_cell_size=8
disc_grid=16

In [5]:
n_images = image_list.shape[0]

In [6]:
hog_list = []
for i in tqdm_notebook(range(n_images), desc="Images"):
    image = image_list[i,:,:,:]
    hog_list.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid))
    
n_features = hog_list[0].size
X_hog = np.array(hog_list).reshape((n_images, n_features)) 


# Data separation

In [7]:
indices = np.random.permutation(X_hog.shape[0])
training_idx, test_idx = indices[:int(0.8*n_train)], indices[int(0.8*n_train):]

X_sample = X_hog[training_idx, :]
n_sample = X_sample.shape[0]
Y_sample = Y_train[training_idx,:]
Y_labels_sample = Y_labels_train[training_idx]

X_test = X_hog[test_idx, :]
n_test = X_test.shape[0]
Y_labels_test = Y_labels_train[test_idx]

# Training

In [28]:
kernel_type = "polynomial"
classifier_type = "svm"
degree = 4
constant = 5
lamb = 0.01

## Kernel choice

In [29]:
K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, degree=degree, constant=constant)

In [30]:
K_sample.shape

(4000, 4000)

## Classifier choice

In [31]:
alpha = np.zeros((n_classes, n_sample))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                           prob_type=classifier_type, lamb=lamb, n_iter=50000)

# Evaluation

### Training error

In [32]:
Y_pred_train = np.zeros((X_sample[:500].shape[0], n_classes))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    Y_pred_train[:, dig] = pred(X_sample, X_sample[:500], alpha[dig, :],
                          kernel_type=kernel_type, degree=degree, constant=constant)


Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
prec = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
print("The precision on the train set is of {}".format(prec))


The precision on the train set is of 1.0


### Testing error

In [33]:
Y_pred = np.zeros((X_test.shape[0], n_classes))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    Y_pred[:, dig] = pred(X_sample, X_test, alpha[dig, :],
                          kernel_type=kernel_type, degree=degree, constant=constant)


Y_labels_pred = np.argmax(Y_pred, axis=1)
prec = np.mean(Y_labels_pred == Y_labels_test)
print("The precision on the test set is of {}".format(prec))


The precision on the test set is of 0.553


In [35]:
1 / (2 * lamb * n_sample)

0.0125

In [37]:
for i in alpha[1]:
    print(i)

0.0
0.0
0.0
0.0
2.09281016909e-12
0.0
0.0
-4.16594097222e-13
0.0
3.29972903546e-14
-3.91268555219e-13
0.0
-3.07376899183e-13
-1.00107533608e-12
0.0
0.0
0.0
9.19959679035e-13
0.0
-4.5123179861e-13
-1.22654414662e-13
0.0
-1.06079607455e-12
-7.80793848223e-13
-3.51588383916e-13
0.0
-2.77909062165e-13
0.0
-1.2150187668e-13
0.0
0.0
-1.33595696497e-12
-7.32502482556e-13
0.0
0.0
1.62467369474e-12
1.21879922568e-13
-2.70801840549e-13
0.0
0.0
1.3167523442e-12
0.0
0.0
0.0
0.0
0.0
0.0
-7.62887260555e-13
1.42716295428e-12
-1.78400188669e-13
-3.24856739095e-14
0.0
0.0
0.0
0.0
2.98190463999e-12
0.0
0.0
0.0
0.0
9.47808424369e-13
0.0
0.0
0.0
0.0
0.0
0.0
-3.92400208802e-13
0.0
-3.83954537617e-13
0.0
1.48875475094e-12
-5.11178515743e-13
0.0
-3.63841762854e-12
-7.09713527737e-13
0.0
0.0
0.0
3.71727362005e-12
-2.44691488848e-13
0.0
-3.41401304874e-13
-2.01724639235e-12
0.0
-6.81309123962e-13
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.1490426092e-12
0.0
-1.29997940778e-12
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-5.

## Grid search from hell

In [ ]:
degrees = [2, 3, 4]
n_degrees = len(degrees)
constants = [0.1, 5, 30]
n_constants = len(constants)
lambs = [0.01, 0.1, 1, 2, 5, 20]
n_lambs = len(lambs)
train_prec = np.zeros((n_degrees, n_constants, n_lambs))
test_prec = np.zeros((n_degrees, n_constants, n_lambs))
for d in tqdm_notebook(range(n_degrees), desc="Degrees"):
    for c in tqdm_notebook(range(n_constants), desc="Constants"):
        # Kernel matrix computation
        K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, degree=degrees[d], constant=constants[c])
        for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
            # Gradient descent
            alpha = np.zeros((n_classes, n_sample))
            for dig in range(n_classes):
                alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                                       prob_type=classifier_type, lamb=lambs[l], n_iter=50000)
            # Precision on train set
            Y_pred_train = np.zeros((X_sample[:500].shape[0], n_classes))
            for dig in range(n_classes):
                Y_pred_train[:, dig] = pred(X_sample, X_sample[:500], alpha[dig, :],
                                      kernel_type=kernel_type, degree=degrees[d], constant=constants[c])


            Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
            train_prec[d, c, l] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
            # Precision on test set
            Y_pred = np.zeros((X_test.shape[0], n_classes))
            for dig in range(n_classes):
                Y_pred[:, dig] = pred(X_sample, X_test, alpha[dig, :],
                                      kernel_type=kernel_type, degree=degrees[d], constant=constants[c])


            Y_labels_pred = np.argmax(Y_pred, axis=1)
            test_prec[d, c, l] = np.mean(Y_labels_pred == Y_labels_test)

In [ ]:
test_prec

In [ ]:
train_prec

# Prediction

In [ ]:
X_eval = load_images_resized(type="test")
X_eval = X_eval - X_eval.min(axis=0)
X_eval = X_eval / X_eval.max(axis=0)
n_eval = X_eval.shape[0]
X_eval2 = np.zeros((2000,63,63,3))
for i in range(3):
    X_eval2[:,:,:,i] = X_eval
image_list_eval = X_eval2

# Visual features for submission
hog_list_eval = []
for i in tqdm_notebook(range(n_eval), desc="Submission images"):
    image = image_list_eval[i,:,:,:]
    hog_list_eval.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid))
    
n_features = hog_list_eval[0].size
X_hog_eval = np.array(hog_list_eval).reshape((n_eval, n_features)) 

In [ ]:
Y_eval = np.zeros((n_eval, n_classes))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    Y_eval[:, dig] = pred(X_sample, X_hog_eval, alpha[dig, :],
                          kernel_type=kernel_type, degree=degree, constant=constant)


Y_labels_eval = np.argmax(Y_eval, axis=1)


# Submission
labels_to_csv(Y_labels_eval, kernel=kernel_type, algo="svm_hog",user="Zac_big_images")